# Statistical tests 
In this notebook, we perform statistical tests to verify if the model with parameters after Random Search tuning is better than:
* model with with parameters after Bayes Search tuning 

## Table of contents
* [Libraries & packages](#lib)
* [Data](#data)
* [Results comparison](#res)
* [Convergence comparison](#con)

## **Libraries and packages** <a class="anchor" id="lib"></a>

In [3]:
# read data
import numpy as np
import pandas as pd

# wilcoxon test
from scipy.stats import wilcoxon, shapiro, ttest_rel

## **Data** <a class="anchor" id="data"></a>

To start with, we load:
- random search results,
- bayes search results,
- default hyperparams results.

Each array in folder 'results' consists of 15 (5 datasets * 3 models) elements - arrays with detailed results.

In [4]:
def unpack_results(path_to_file):
    results = np.load(path_to_file, allow_pickle=True)
    datasets = []

    all_columns = set()
    for result in results:
        all_columns.update(result.keys())
    all_columns = list(all_columns)

    for i, result in enumerate(results):
        df = pd.DataFrame(result)
        df = df.reindex(columns=all_columns)
        datasets.append(df)
        
    all_datasets = pd.concat(datasets)
    return all_datasets

In [5]:
rand = unpack_results('../results/random_search_tuning.npy')
bayes = unpack_results('../results/bayes_search_tuning.npy')
default = unpack_results('../results/default_hyperparameter.npy')

Let's examine some of them.

In [6]:
rand.tail(6)

,param_xgb__reg_lambda,split1_test_score,std_score_time,mean_test_score,param_knn__metric,split3_test_score,params,rank_test_score,split4_test_score,mean_fit_time,...,param_logreg__solver,mean_score_time,param_xgb__learning_rate,dataset,param_logreg__max_iter,param_logreg__penalty,split2_test_score,param_knn__weights,param_knn__n_neighbors,split0_test_score
94,NaN,0.616,0.003381,0.6310,manhattan,0.636,"{'knn__metric': 'manhattan', 'knn__n_neighbors...",23,0.634,0.019836,...,NaN,0.088980,NaN,higgs,NaN,NaN,0.636,uniform,91.0,0.633
95,NaN,0.591,0.002093,0.5884,euclidean,0.593,"{'knn__metric': 'euclidean', 'knn__n_neighbors...",92,0.587,0.014696,...,NaN,0.023782,NaN,higgs,NaN,NaN,0.571,distance,7.0,0.600
96,NaN,0.601,0.001127,0.5966,euclidean,0.600,"{'knn__metric': 'euclidean', 'knn__n_neighbors...",80,0.596,0.015474,...,NaN,0.028971,NaN,higgs,NaN,NaN,0.578,distance,28.0,0.608
97,NaN,0.600,0.003302,0.6038,minkowski,0.611,"{'knn__metric': 'minkowski', 'knn__n_neighbors...",68,0.610,0.017254,...,NaN,0.043820,NaN,higgs,NaN,NaN,0.592,uniform,47.0,0.606
98,NaN,0.615,0.006869,0.6316,manhattan,0.640,"{'knn__metric': 'manhattan', 'knn__n_neighbors...",20,0.625,0.014476,...,NaN,0.087015,NaN,higgs,NaN,NaN,0.638,uniform,77.0,0.640
99,NaN,0.590,0.004837,0.5980,euclidean,0.614,"{'knn__metric': 'euclidean', 'knn__n_neighbors...",77,0.607,0.014933,...,NaN,0.041920,NaN,higgs,NaN,NaN,0.576,uniform,32.0,0.603


In [7]:
MODELS = ('xgb', 'logreg', 'knn')
METHODS = ('random', 'bayes', 'default')
DATASETS = ('higgs', 'kc1', 'magic', 'mushroom', 'ozone')

## **Results comparison** <a class="anchor" id="res"></a>

For each model, the test will be performed:

H0: There is *no* significant difference between best mean test scores  
H1: There is a significant difference between best mean test scores 

with significance level alpha = 0.05.

Let X be the best mean test score reached via one of the methods and Y - via another one.

Firstly, the Shapiro-Wilk test will be performed to determine if X-Y has gaussian distribution. If yes, then t-test is performed. Otherwise, we use Wilcoxon test.

Necessary functions:

In [8]:
def prepare_vector_results(model_name, method_name):
    ''' 
    Returns array of best scores for each dataset, in alphabetical order of datasets.
    '''
    if method_name == 'random':
        df = rand
    elif method_name == 'bayes':
        df = bayes
    elif method_name == 'default':
        df = default
    
    df = df[df['model'] == model_name]
    df_ = df[['dataset', 'mean_test_score']].groupby('dataset').max().reset_index().sort_values(by='dataset')

    return df_['mean_test_score'].values


In [9]:
def calculate_mean_test_scores_for_default():
    ''' 
    Returns dataframe with mean test scores for each model and dataset, on default data frame. 
    '''
    df = default
    df_ = df[['model', 'dataset', 'test_score']].groupby(['model','dataset']).mean().reset_index().sort_values(by='dataset')
    df_.rename(columns={'test_score':'mean_test_score'}, inplace=True)
    return df_


In [10]:
default = calculate_mean_test_scores_for_default()

In [11]:
def run_tests_res():
    alpha = 0.05
    results = []

    for model in MODELS:
        for method_pair in [('random', 'bayes'), ('random', 'default'), ('bayes', 'default')]:
            meth_1 = method_pair[0]
            meth_2 = method_pair[1]
            X = prepare_vector_results(model, meth_1)
            Y = prepare_vector_results(model, meth_2)

            diff = X - Y
            diff = diff[~np.isnan(diff)]
            
            shapiro_p = shapiro(diff, nan_policy='omit').pvalue

            if shapiro_p > alpha:
                t = "t-test"
                p = ttest_rel(X, Y).pvalue
            else:
                t = "wilcoxon test"
                p = wilcoxon(X, Y, alternative='two-sided').pvalue
            
            results.append({
                'model': model,
                'method 1': meth_1,
                'method 2': meth_2,
                'test': t,
                'p_value': p,
                'null_hypothesis': 'rejected' if p < alpha else 'not rejected'
            })

    results_df = pd.DataFrame(results)
    return results_df

In [12]:
run_tests_res()

,model,method 1,method 2,test,p_value,null_hypothesis
0,xgb,random,bayes,t-test,0.845555,not rejected
1,xgb,random,default,t-test,0.107512,not rejected
2,xgb,bayes,default,t-test,0.413406,not rejected
3,logreg,random,bayes,t-test,0.849031,not rejected
4,logreg,random,default,t-test,0.023207,rejected
5,logreg,bayes,default,t-test,0.752208,not rejected
6,knn,random,bayes,t-test,0.930449,not rejected
7,knn,random,default,t-test,0.099877,not rejected
8,knn,bayes,default,t-test,0.463025,not rejected


## **Convergence comparison** <a class="anchor" id="con"></a>

For each model, for each dataset the test will be performed:

H0: There is *no* significant difference between the convergence speed  
H1: There is a significant difference between the convergence speed

with significance level alpha = 0.05.

Let X be the accumulative best test score reached via Random Search and Y - via Bayes Search.

Firstly, the Shapiro-Wilk test will be performed to determine if X-Y has gaussian distribution. If yes, then t-test is performed. Otherwise, we use Wilcoxon test.

In [13]:
def prepare_vector_conv(model_name, method_name, dataset_name):
    ''' 
    Returns array of cumulative best scores for each dataset, in alphabetical order of datasets.
    '''
    if method_name == 'random':
        df = rand
    elif method_name == 'bayes':
        df = bayes
    
    result =  df[(df['model'] == model_name) & (df['dataset'] == dataset_name)]['mean_test_score'].cummax()

    if method_name == 'random':
        result = result[:60]

    return result.values

In [14]:
def run_tests_conv():
    alpha = 0.05
    results = []

    for model in MODELS:
        for dataset in DATASETS:
            X = prepare_vector_conv(model, 'random', dataset)
            Y = prepare_vector_conv(model, 'bayes', dataset)

            diff = X - Y
            diff = diff[~np.isnan(diff)]
            
            shapiro_p = shapiro(diff, nan_policy='omit').pvalue

            if shapiro_p > alpha:
                t = "t-test"
                p = ttest_rel(X, Y).pvalue
            else:
                t = "wilcoxon test"
                p = wilcoxon(X, Y, alternative='two-sided').pvalue
            
            results.append({
                'model': model,
                'dataset': dataset,
                'test': t,
                'p_value': p,
                'null_hypothesis': 'rejected' if p < alpha else 'not rejected'
            })

    results_df = pd.DataFrame(results)
    return results_df

In [15]:
run_tests_conv()

,model,dataset,test,p_value,null_hypothesis
0,xgb,higgs,wilcoxon test,8.879560e-12,rejected
1,xgb,kc1,wilcoxon test,7.076742e-12,rejected
2,xgb,magic,wilcoxon test,1.195110e-11,rejected
3,xgb,mushroom,wilcoxon test,2.530149e-06,rejected
4,xgb,ozone,wilcoxon test,7.534773e-13,rejected
5,logreg,higgs,wilcoxon test,5.987643e-13,rejected
6,logreg,kc1,wilcoxon test,1.099584e-13,rejected
7,logreg,magic,wilcoxon test,9.592416e-12,rejected
8,logreg,mushroom,wilcoxon test,9.301841e-13,rejected
9,logreg,ozone,wilcoxon test,1.435184e-11,rejected


Thus, we will examine, if cumulative best score for Bayes Search is significantly greater than the one achieved by Random Search.

Let X be the accumulative best test score reached via Random Search and Y - via Bayes Search.


For each model, for each dataset the test will be performed:

H0: median(X-Y) <= 0  (X is lower or equal than Y)  
H1: median(X-Y) > 0 (X is greater than Y)

with significance level alpha = 0.05.



In [18]:
def run_tests_conv_geq():
    alpha = 0.05
    results = []

    for model in MODELS:
        for dataset in DATASETS:
            X = prepare_vector_conv(model, 'random', dataset)
            Y = prepare_vector_conv(model, 'bayes', dataset)

            p = wilcoxon(X, Y, alternative='greater').pvalue
            
            results.append({
                'model': model,
                'dataset': dataset,
                'p_value': p,
                'null_hypothesis': 'rejected' if p < alpha else 'not rejected',
                'interpretation': 'Random Search' if p < alpha else 'Bayes Search'
            })

    results_df = pd.DataFrame(results)
    return results_df

In [19]:
run_tests_conv_geq()

,model,dataset,p_value,null_hypothesis,interpretation
0,xgb,higgs,1.000000e+00,not rejected,Bayes Search
1,xgb,kc1,3.538371e-12,rejected,Random Search
2,xgb,magic,1.000000e+00,not rejected,Bayes Search
3,xgb,mushroom,9.999987e-01,not rejected,Bayes Search
4,xgb,ozone,3.767386e-13,rejected,Random Search
5,logreg,higgs,1.000000e+00,not rejected,Bayes Search
6,logreg,kc1,5.497918e-14,rejected,Random Search
7,logreg,magic,1.000000e+00,not rejected,Bayes Search
8,logreg,mushroom,1.000000e+00,not rejected,Bayes Search
9,logreg,ozone,7.175920e-12,rejected,Random Search
